In [2]:
from math import exp
from mxnet import gluon
from mxnet import autograd
from mxnet import nd
from mxnet import image
from mxnet.gluon import nn
import mxnet as mx
import numpy as np
from time import time
import matplotlib.pyplot as plt

# 多层感知机 --- 从0开始

前面我们介绍了包括线性回归和多类逻辑回归的数个模型，它们的一个共同点是全是只含有一个输入层，一个输出层。这一节我们将介绍多层神经网络，就是包含至少一个隐含层的网络。

## 数据获取

我们继续使用 FashionMNIST 数据集。

In [3]:
batch_size = 256
root = 'E:/Data/MXNet/fashion_mnist'

def transform(data, label):
    return data.astype('float32')/255, label.astype('float32')
mnist_train = gluon.data.vision.FashionMNIST(train=True, transform=transform, root= root)
mnist_test = gluon.data.vision.FashionMNIST(train=False, transform=transform, root= root)

train_data = gluon.data.DataLoader(mnist_train, batch_size, shuffle=True)
test_data = gluon.data.DataLoader(mnist_test, batch_size, shuffle=False)

c:\anaconda3\lib\site-packages\mxnet\gluon\data\vision.py:118: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  label = np.fromstring(fin.read(), dtype=np.uint8).astype(np.int32)
c:\anaconda3\lib\site-packages\mxnet\gluon\data\vision.py:122: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  data = np.fromstring(fin.read(), dtype=np.uint8)


## 多层感知机

多层感知机与前面介绍的*多类逻辑回归*非常类似，主要的区别是我们在输入层和输出层之间插入了一个到多个隐含层。

这里我们定义一个只有一个隐含层的模型，这个隐含层输出 $256$ 个节点。

In [3]:
num_inputs = 28*28
num_outputs = 10

num_hidden = 256
weight_scale = .01

W1 = nd.random_normal(shape=(num_inputs, num_hidden), scale=weight_scale)
b1 = nd.zeros(num_hidden)

W2 = nd.random_normal(shape=(num_hidden, num_outputs), scale=weight_scale)
b2 = nd.zeros(num_outputs)

params = [W1, b1, W2, b2]

for param in params:
    param.attach_grad()

## 激活函数

如果我们就用线性操作符来构造多层神经网络，那么整个模型仍然只是一个线性函数。这是因为

$$\hat{y} = X \cdot W_1 \cdot W_2 = X \cdot W_3 $$

这里$W_3 = W_1 \cdot W_2$。为了让我们的模型可以拟合非线性函数，我们需要在层之间插入非线性的激活函数。这里我们使用ReLU

$$\textrm{rel}u(x)=\max(x, 0)$$

In [4]:
def relu(X):
    return nd.maximum(X, 0)

## 定义模型

我们的模型就是将层（全连接）和激活函数（Relu）串起来：

In [12]:
def net(X):
    X = X.reshape((-1, num_inputs))
    h1 = relu(nd.dot(X, W1) + b1)
    output = nd.dot(h1, W2) + b2
    return output

def SGD(params, lr):
    for param in params:
        param[:] -= lr * param.grad

## Softmax和交叉熵损失函数

在多类Logistic回归里我们提到分开实现Softmax和交叉熵损失函数可能导致数值不稳定。这里我们直接使用Gluon提供的函数

In [13]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

## 训练

训练跟之前一样。

In [16]:
def accuracy(output, label):
    return nd.mean(output.argmax(axis=1)==label).asscalar()

def evaluate_accuracy(data_iterator, net):
    acc = 0.
    for data, label in data_iterator:
        output = net(data)
        acc += accuracy(output, label)
    return acc / len(data_iterator)

learning_rate = .5

for epoch in range(10):
    train_loss = 0.
    train_acc = 0.
    for data, label in train_data:
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        SGD(params, learning_rate/batch_size)

        train_loss += nd.mean(loss).asscalar()
        train_acc += accuracy(output, label)

    test_acc = evaluate_accuracy(test_data, net)
    print("Epoch %d. Loss: %f, Train acc %f, Test acc %f" % (
        epoch, train_loss/len(train_data),
        train_acc/len(train_data), test_acc))

Epoch 0. Loss: 0.299955, Train acc 0.888636, Test acc 0.879883
Epoch 1. Loss: 0.295771, Train acc 0.891728, Test acc 0.888770
Epoch 2. Loss: 0.289865, Train acc 0.892221, Test acc 0.878516
Epoch 3. Loss: 0.282838, Train acc 0.894952, Test acc 0.888281
Epoch 4. Loss: 0.277365, Train acc 0.897390, Test acc 0.887500
Epoch 5. Loss: 0.273307, Train acc 0.899324, Test acc 0.890332
Epoch 6. Loss: 0.268238, Train acc 0.900537, Test acc 0.887012
Epoch 7. Loss: 0.262066, Train acc 0.902094, Test acc 0.890918
Epoch 8. Loss: 0.255037, Train acc 0.904671, Test acc 0.886230
Epoch 9. Loss: 0.252222, Train acc 0.905967, Test acc 0.888477


## 总结

可以看到，加入一个隐含层后我们将精度提升了不少。

## 练习

- 我们使用了 `weight_scale` 来控制权重的初始化值大小，增大或者变小这个值会怎么样？
- 尝试改变 `num_hiddens` 来控制模型的复杂度
- 尝试加入一个新的隐含层

**吐槽和讨论欢迎点**[这里](https://discuss.gluon.ai/t/topic/739)